In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display

In [2]:
class ExerciseTracker:
    def __init__(self, df):
        self.df = df

    def log_exercise(self, name, weight, reps, date=datetime.now().date()):
        new_data = {
            'Exercise': [name],
            'Weight': [weight],
            'Reps': [reps],
            'Date': [pd.to_datetime(date)]  # convert to datetime
        }
        new_df = pd.DataFrame(new_data)
        self.df = pd.concat([self.df, new_df], ignore_index=True)

    def calculate_1rm(self, name):
        exercise_data = self.df[self.df['Exercise'] == name]
        if not exercise_data.empty:
            weight = exercise_data['Weight'].iloc[-1]  # get the latest weight
            reps = exercise_data['Reps'].iloc[-1]  # get the latest reps
            return weight + (weight * reps * 0.0333)
        else:
            print(f"No data for exercise {name}")
            return None
    def plot_1rm(self, name):
        exercise_data = self.df[self.df['Exercise'] == name]
        if not exercise_data.empty:
            dates = exercise_data['Date'].dt.strftime('%Y-%m-%d')  # convert to string format
            weights = exercise_data['Weight']
            reps = exercise_data['Reps']
            estimated_1rm = weights + (weights * reps * 0.0333)

            plt.figure(figsize=(10, 6))
            plt.scatter(dates, estimated_1rm)
            plt.plot(dates, estimated_1rm)
            plt.title(f'Estimated 1RM for {name} Over Time')
            plt.xlabel('Date')
            plt.ylabel('Estimated 1RM (kg)')
            plt.xticks(rotation=45)  # rotate the x-axis labels for better readability

            # Make x-axis display only dates that have data
            ax = plt.gca()  # get current axis
            ax.set_xticks(dates)  # set x-axis ticks to dates that have data

            plt.tight_layout()  # adjust the layout for better readability
            plt.show()
        else:
            print(f"No data for exercise {name}")

In [3]:
df = pd.read_csv('../data/exercise_log.csv', parse_dates=['Date'])
tracker = ExerciseTracker(df)

In [4]:
# Get unique exercise names from the DataFrame
exercise_names = df['Exercise'].unique()

# Create input widgets
exercise_name_input = widgets.Dropdown(description='Exercise Name:', options=exercise_names)
weight_input = widgets.FloatText(description='Weight (kg):', value=90)
reps_input = widgets.IntText(description='Reps:', value=3)
date_input = widgets.DatePicker(description='Date:', value=datetime.now().date())

# Create output widgets
output = widgets.Output()
plot_output = widgets.Output()

# Create button and define its click event handler
button = widgets.Button(description='Log Exercise')

def log_exercise(button):
    with output, plot_output:
        exercise_name = exercise_name_input.value
        weight = weight_input.value
        reps = reps_input.value
        date = date_input.value.strftime('%Y-%m-%d')

        tracker.log_exercise(exercise_name, weight, reps, date)
        result = tracker.calculate_1rm(exercise_name)
        if result:
            print(f"The estimated 1RM for {exercise_name} is {result} kg")
        tracker.plot_1rm(exercise_name)

# Associate the button click event handler with the button
button.on_click(log_exercise)

In [5]:
# Display the widgets and output
display(exercise_name_input, weight_input, reps_input, date_input, button)
display(output, plot_output)

Dropdown(description='Exercise Name:', options=('Front Squat', 'Snatch', 'Back Squat', 'Power Snatch', 'Deadli…

FloatText(value=90.0, description='Weight (kg):')

IntText(value=3, description='Reps:')

DatePicker(value=datetime.date(2023, 6, 16), description='Date:', step=1)

Button(description='Log Exercise', style=ButtonStyle())

Output()

Output()